In [133]:
import numpy as np
import nltk
import pandas as pd
import re
import random
import csv
import pickle
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')

# Load and clean raw data 

In [22]:
csv = pd.read_csv('sample_file.txt')

In [27]:
csv.head()

,aspect_ratio,audio_codec,audio_sample_rate,backup_location,ccnum,closed_captioning,collection,color,contributor,date,...,subject,thumbs,times,title,tuner,uploader,utc_offset,video_codec,year,text
0,4:3,mp2,48000,ia905706_34,cc5,yes,TV-FOXNEWS|tvarchive|tvnews,color,FOXNEWS,2013-09-14,...,joe; colorado; julie; wayne; eric; usaa; washi...,"45,75,105,135,165,195,225,255,285,315,345,375,...","1,3,4,4,6,7,11,12,18,18,20,21,22,24,25,27,28,3...","Cashin' In : FOXNEWS : September 14, 2013 11:3...",Port 1236,tv@archive.org,-400,mpeg2video,2013,thank you for watching. have a wonderful weeke...
1,4:3,mp2,48000,ia905606_23,cc5,yes,TV-FOXNEWS|tvarchive|tvnews,color,FOXNEWS,2013-02-12,...,christopher dorner; los angeles; california; l...,"45,75,105,135,165,195,225,255,285,315,345,375,...","0,2,7,9,11,12,13,16,18,20,24,29,31,36,37,42,43...",Your World With Neil Cavuto : FOXNEWS : Februa...,Port 1236,tv@archive.org,-500,mpeg2video,2013,let the good times roll. they had a mini carna...
2,4:3,mp2,48000,ia905606_24,cc5,yes,TV-FOXNEWS|tvarchive|tvnews,color,FOXNEWS,2013-02-13,...,rubio; georgia; florida; heather; obama; marco...,"45,75,105,135,165,195,225,255,285,315,345,375,...","21,22,23,25,26,27,29,32,32,33,34,35,38,40,41,4...","FOX and Friends First : FOXNEWS : February 13,...",Port 1236,tv@archive.org,-500,mpeg2video,2013,>> good morning everyone. i am patti ann brown...
3,4:3,mp2,48000,ia905703_13,cc5,yes,TV-FOXNEWS|tvarchive|tvnews,color,FOXNEWS,2013-07-09,...,zimmerman; obama; mary katharine; bill o'reill...,"45,75,105,135,165,195,225,255,285,315,345,375,...","0,2,2,3,4,6,8,8,8,9,10,11,13,17,20,21,23,24,25...","The O'Reilly Factor : FOXNEWS : July 8, 2013 1...",Port 1236,tv@archive.org,-400,mpeg2video,2013,>> thank you. and thanks for joining us tonigh...
4,4:3,mp2,48000,ia905609_14,cc5,yes,TV-FOXNEWS|tvarchive|tvnews,color,FOXNEWS,2013-04-15,...,irs; angie; warren buffett; tennessee; califor...,"45,75,105,135,165,195,225,255,285,315,345,375,...","11,14,18,20,20,22,25,27,28,29,31,35,37,38,39,4...","Stossel : FOXNEWS : April 15, 2013 1:00am-2:00...",Port 1236,tv@archive.org,-400,mpeg2video,2013,[laughter] >> the clock is ticking. taxes are ...


In [28]:
text = csv[['text']]

In [29]:
text.head()

,text
0,thank you for watching. have a wonderful weeke...
1,let the good times roll. they had a mini carna...
2,>> good morning everyone. i am patti ann brown...
3,>> thank you. and thanks for joining us tonigh...
4,[laughter] >> the clock is ticking. taxes are ...


In [72]:
music_note = text.iloc[0][0][140]
music_note

'♪'

In [89]:
def data_clean(data):
    # remove >> 
    data = data.split(">>")
    data = ''.join(data)
    # remove \
    data = data.split("\\")
    data = ''.join(data)
    # remove music notes
    data = data.split(music_note)
    data = ''.join(data)
    # remove speaker names 
    data = re.sub(r'\w+:\s?','',data)
    # remove [ some note about what's happening ]
    data = re.sub(' \[ [\w ]+ \] \s?','',data)
    return data
    

In [90]:
all_text = []
for i in range(len(text)):
    all_text.append(str(text.iloc[i][0]))
    
all_text = ''.join(all_text)

In [91]:
clean_data = data_clean(all_text)

In [92]:
len(clean_data)

1736288

In [93]:
# # Uncomment to write to file again 
# with open(r"clean_text.txt", "w") as f:
#     f.write(clean_data)

# Create words to sentence dataset

In [94]:
sents = nltk.sent_tokenize(clean_data)

In [95]:
sents[:15]

['thank you for watching.',
 'have a wonderful weekend.',
 'keep it here.',
 'the number one business block continues with eric bolling and "cashin\' in."',
 'a warning sign for all americans.',
 'obamacare may be hazardous to your health plan.',
 "don't believe this billboard by the heritage foundation?",
 'how about this job creator?',
 "it's the latest company showing just how hazardous the law is for workers.",
 'plus, super power showdown over syria, how the crisis over there could prove we are losing the economic superpower status everywhere.',
 'then, desperate times call for desperate measures.',
 'first they supported low-wage worker now they are looking for any worker.',
 'how the boom could be a bust for jobs.',
 '"cashin\' in."',
 'working hard for you right now.']

In [125]:
def remove_pos(words, kept_pos=['NOUN', 'VERB', 'ADV', 'ADJ', 'PROPN', 'INTJ', 'NUM']):
    words_pos = nltk.pos_tag(words, tagset='universal')
    
    for i, word in enumerate(words):
        if words_pos[i][1] not in kept_pos:
            del words[i]
            
    return words

def scramble_order(words, fraction_scrambled=0.1):
    num = random.random()
    if num<fraction_scrambled:
        random.shuffle(words)
        
    return words
    

In [126]:
sw_dict = {}
for sent in sents:
    sw_dict[sent] = scramble_order(remove_pos(nltk.word_tokenize(sent)))

In [127]:
sw_dict

{'thank you for watching.': ['thank', 'for', '.'],
 'have a wonderful weekend.': ['have', 'wonderful', 'weekend', '.'],
 'keep it here.': ['keep', 'here', '.'],
 'the number one business block continues with eric bolling and "cashin\' in."': ['number',
  'one',
  'business',
  'block',
  'continues',
  'with',
  'bolling',
  'and',
  '``',
  "'",
  '.',
  "''"],
 'a warning sign for all americans.': ['warning', 'sign', 'for', 'americans'],
 'obamacare may be hazardous to your health plan.': ['obamacare',
  'may',
  'be',
  'hazardous',
  'your',
  'plan',
  '.'],
 "don't believe this billboard by the heritage foundation?": ['do',
  "n't",
  'believe',
  'billboard',
  'by',
  'heritage',
  '?'],
 'how about this job creator?': ['how', 'this', 'creator', '?'],
 "it's the latest company showing just how hazardous the law is for workers.": ["'s",
  'the',
  'company',
  'showing',
  'just',
  'how',
  'hazardous',
  'the',
  'law',
  'for',
  'workers',
  '.'],
 'plus, super power showdow

In [132]:
# # Uncomment to write to csv 
# w = csv.writer(open("sw_data.csv", "w"), delimiter=',', lineterminator='\n')
# for key, val in sw_dict.items():
#     w.writerow([key, val])

In [134]:
# # Uncomment to write to pickle 
# f = open("sw_dict.pkl","wb")
# pickle.dump(sw_dict,f)
# f.close()